In [8]:
import uuid

name = 'hello world'
new_id = uuid.uuid4()
new_name = {'id': str(new_id),'name': str(name)}
new_name

{'id': '2fcea5a2-8921-4a9d-9653-9fac36f75cf1', 'name': 'hello world'}

In [42]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, UserDelegationKey, generate_container_sas, ContainerSasPermissions
import datetime

account_url = "https://vikpwnywivhg2models.blob.core.windows.net"
container_name = "modified-vikpwnywivhg2"
default_credential = DefaultAzureCredential()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient(account_url, credential=default_credential)
blob_service_client

In [32]:
container_client = blob_service_client.get_container_client(container=container_name)
container_client.get_container_properties()


{'name': 'modified-vikpwnywivhg2', 'last_modified': datetime.datetime(2023, 12, 24, 20, 19, 17, tzinfo=datetime.timezone.utc), 'etag': '"0x8DC04BD9AC2AE93"', 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'public_access': None, 'has_immutability_policy': False, 'deleted': None, 'version': None, 'has_legal_hold': False, 'metadata': {}, 'encryption_scope': <azure.storage.blob._models.ContainerEncryptionScope object at 0x7fffd31ad3d0>, 'immutable_storage_with_versioning_enabled': False}

In [36]:
def request_user_delegation_key(blob_service_client: BlobServiceClient) -> UserDelegationKey:
    # Get a user delegation key that's valid for 1 day
    delegation_key_start_time = datetime.datetimedef create_user_delegation_sas_container(self, container_client: ContainerClient, user_delegation_key: UserDelegationKey):
    # Create a SAS token that's valid for one day, as an example
    start_time = datetime.datetime.now(datetime.timezone.utc)
    expiry_time = start_time + datetime.timedelta(days=1)

    sas_token = generate_container_sas(
        account_name=container_client.account_name,
        container_name=container_client.container_name,
        user_delegation_key=user_delegation_key,
        permission=ContainerSasPermissions(read=True),
        expiry=expiry_time,
        start=start_time
    )

    return sas_token.now(datetime.timezone.utc)
    delegation_key_expiry_time = delegation_key_start_time + datetime.timedelta(days=1)

    user_delegation_key = blob_service_client.get_user_delegation_key(
        key_start_time=delegation_key_start_time,
        key_expiry_time=delegation_key_expiry_time
    )

    return user_delegation_key

In [47]:
def create_user_delegation_sas_container(container_client: ContainerClient, user_delegation_key: UserDelegationKey):
    # Create a SAS token that's valid for one day, as an example
    start_time = datetime.datetime.now(datetime.timezone.utc)
    expiry_time = start_time + datetime.timedelta(hours=1)

    sas_token = generate_container_sas(
        account_name=container_client.account_name,
        container_name=container_client.container_name,
        user_delegation_key=user_delegation_key,
        permission=ContainerSasPermissions(read=True, list=True),
        expiry=expiry_time,
        start=start_time
    )

    return sas_token

In [48]:
user_delegation_key = request_user_delegation_key(blob_service_client)
user_delegation_key

In [49]:
sas_token = create_user_delegation_sas_container(container_client, user_delegation_key)
sas_token

'st=2023-12-24T21%3A37%3A10Z&se=2023-12-24T22%3A37%3A10Z&sp=rl&sv=2023-11-03&sr=c&skoid=b57b8480-78cd-4a24-bb01-c9ec1ff4d5e2&sktid=8103a899-c0d2-44bf-bf1d-0580a25bcb7c&skt=2023-12-24T21%3A37%3A08Z&ske=2023-12-25T21%3A37%3A08Z&sks=b&skv=2023-11-03&sig=2cA2h/NK3elYykivfM5U2mCtscCn1eDa31mrNl%2BH/A4%3D'

In [50]:
sas_url = f"{container_client.url}?{sas_token}"
sas_url

'https://vikpwnywivhg2models.blob.core.windows.net/modified-vikpwnywivhg2?st=2023-12-24T21%3A37%3A10Z&se=2023-12-24T22%3A37%3A10Z&sp=rl&sv=2023-11-03&sr=c&skoid=b57b8480-78cd-4a24-bb01-c9ec1ff4d5e2&sktid=8103a899-c0d2-44bf-bf1d-0580a25bcb7c&skt=2023-12-24T21%3A37%3A08Z&ske=2023-12-25T21%3A37%3A08Z&sks=b&skv=2023-11-03&sig=2cA2h/NK3elYykivfM5U2mCtscCn1eDa31mrNl%2BH/A4%3D'

In [51]:
container_client_sas = ContainerClient.from_container_url(container_url=sas_url)

In [52]:
blob_names = list(container_client_sas.list_blob_names())
blob_names

['5fe9a991-34f3-409a-a6d5-376dcf875493.json',
 'ea22ec66-5a52-4da1-be92-d10330a1360d.json']